**Entendendo o H3**

O h3 location_id = 8aa8a02c355ffff é um identificador único de uma célula em um determinado nível de resolução na grade H3. Para entender o que ele significa, vamos decompor o índice conforme descrito:
- Bits reservados e configurados para 0 (1 bit): O primeiro bit do índice é reservado e sempre será 0.
- Modo do H3 Cell (4 bits): A seguir, há 4 bits que indicam o modo. O modo para um H3 Cell index é 1, o que identifica o índice como um hexágono ou pentágono.
- Bits reservados e configurados para 0 (3 bits): Estes 3 bits são reservados e sempre são 0.
- Resolução da célula (4 bits): Depois, há 4 bits que indicam a resolução da célula. A resolução pode variar de 0 a 15. A resolução determina o tamanho do hexágono: *quanto maior a resolução, menor o hexágono*.
- Base cell (7 bits): A base cell é identificada pelos próximos 7 bits, variando de 0 a 121. Cada base cell corresponde a um hexágono de nível 0 na grade H3.
- Subsequente de dígitos (45 bits no total): Finalmente, temos 3 bits para cada dígito subsequente que identificam as células em níveis de resolução maiores. Para resoluções não utilizadas, os bits são preenchidos com 7.

O valor hexadecimal "8aa8a02c355ffff" convertido para binário é:
0000100010101010100010100000001011000011010101011111111111111111
De acordo com a estrutura do H3 Cell index:
- 1 bit reservado e configurado para 0: 0
- 4 bits para o modo: 0001 (indica o modo H3 Cell)
- 3 bits reservados e configurados para 0: 000
- 4 bits para a resolução da célula: 1010 (resolução 10)
- 7 bits para a base cell: 1010101 (base cell 85)
- 3 bits para cada dígito subsequente até a resolução 15(não usada)

Resumo dos componentes:
- Modo: 1 (H3 Cell)
- Resolução: 10
- Base Cell: 85
- Dígitos subsequentes: 000, 000, 010, 110, 000, 011, 010, 101, 101, 111, 111 (até a resolução 10)

Resolução 10:
Quando dizemos que a resolução é 10, estamos nos referindo a um hexágono que é muito mais detalhado do que aqueles em resoluções mais baixas. Especificamente, a cada aumento de resolução, um hexágono é subdividido em 7 hexágonos de nível mais alto, tornando-os menores.

Este tamanho de hexágono é utilizado para mapear áreas relativamente pequenas com alta precisão geoespacial, como bairros ou pequenas porções de uma cidade. É útil para aplicações que requerem análise espacial detalhada, como rastreamento de mobilidade humana ou análise de padrões urbanos.

H3 Index Inspector
https://observablehq.com/@nrabinowitz/h3-index-inspector?collection=@nrabinowitz/h3

"8aa8a02c355ffff"
- cellArea	15,522.990 m2
- avg edge length	77.336366 m

"8aa884084077fff"
- cellArea	15,843.914 m2
- avg edge length	78.121905 m

centro do RIO: "8aa8a06a0c6ffff"

In [4]:
import pandas as pd
# Carregar o arquivo com os índices H3
file_path = '/Users/andreza/mobility_economics_climate/data/h3_visitation_daily_RJ.csv'
df = pd.read_csv(file_path)

In [12]:
def h3_resolution(h3_index):
    """Função para extrair a resolução de um índice H3."""
    # Converte o índice H3 de hexadecimal para binário
    h3_bin = bin(int(h3_index, 16))[2:].zfill(64)
    # Os bits 16 a 19 (baseado em zero) são a resolução (posição 12 a 15 em notação zero-indexed)
    resolution_bits = h3_bin[12:16]
    return int(resolution_bits, 2)

# Aplicando a função à coluna location_id
df['resolucao'] = df['location_id'].apply(h3_resolution)

# Verifica se todos os índices têm a mesma resolução
resolutions = df['resolucao'].unique()
print("Resoluções encontradas:", resolutions)

# Salvar o DataFrame com as resoluções em um novo arquivo CSV
#df.to_csv('output_resolutions.csv', index=False)

# verificar se todas as resoluções são iguais a 10:
if all(df['resolucao'] == 10):
    print("Todos os índices têm resolução 10.")
else:
    print("Existem índices com resoluções diferentes de 10.")


Resoluções encontradas: [10]
Todos os índices têm resolução 10.


In [7]:
pip install h3

     |████████████████████████████████| 1.0 MB 429 kB/s            
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install geopy

     |████████████████████████████████| 118 kB 2.1 MB/s            
Note: you may need to restart the kernel to use updated packages.


In [11]:
from shapely.geometry import Point
import h3
from geopy.distance import geodesic

# Coordenadas aproximadas da capital Rio de Janeiro
rio_de_janeiro_coords = (-22.9068, -43.1729)  # (latitude, longitude)

# Função para calcular a distância geodésica entre dois pontos
def calculate_distance(point_coords, reference_coords):
    return geodesic(point_coords, reference_coords).meters

# Função para extrair a latitude e longitude do ponto da coluna 'geometry'
def extract_coords(geometry):
    point = Point(map(float, geometry.strip('POINT ()').split()))
    return (point.y, point.x)

# Extrair as coordenadas de cada ponto e calcular a distância para o Rio de Janeiro
df['coords'] = df['geometry'].apply(extract_coords)
df['distance_to_rio'] = df['coords'].apply(lambda x: calculate_distance(x, rio_de_janeiro_coords))

# Identificar o hexágono com a menor distância
closest_hexagon = df.loc[df['distance_to_rio'].idxmin()]

print("O hexágono mais próximo do Rio de Janeiro tem o location_id:", closest_hexagon['location_id'])
print("Distância:", closest_hexagon['distance_to_rio'], "metros")

# salvar o DataFrame atualizado
#df.to_csv('output_with_distances.csv', index=False)


O hexágono mais próximo do Rio de Janeiro tem o location_id: 8aa8a06a0c6ffff
Distância: 65.4906431212114 metros
